# Loads workitems from a query
A notebook to load data from devops, merge and clean the data before storing it to a file for analyzis

In [1]:
import os
import datetime
from pytz import timezone
import pandas as pd
from azure.devops.credentials import BasicAuthentication
from azure.devops.connection import Connection
from azure.devops.v7_1.work_item_tracking.models import Wiql

## Configuration 
auth_token is a personal access token (PAT) stored in a Windows environment variable called AZURE_DEVOPS_PAT

In [2]:
__VERSION__ = "1.0.0"
auth_token = os.environ['AZURE_DEVOPS_PAT']
url ="https://dev.azure.com/skanskanordic/"

increment = "Skanska Sverige IT\\2023\Vinter 2023-4"
area = "Skanska Sverige IT"

after_planning = datetime.datetime(2023,11, 20,0,0, tzinfo=timezone('UTC'))
after_delivery = datetime.datetime(2024, 2, 6,10,0, tzinfo=timezone('UTC'))


In [3]:
from types import SimpleNamespace
context = SimpleNamespace()
context.runner_cache = SimpleNamespace()
context.connection = Connection(base_url=url,creds=BasicAuthentication('PAT', auth_token), user_agent='azure-devops-python-samples/' + __VERSION__)


## Workitems is loaded from devops using a query

In [4]:
# List all tasks that at the current moment is within the increment and areapaths for the team.
desired_ids = list()
all_tasks = pd.DataFrame()
wit_client = context.connection.clients.get_work_item_tracking_client()

wiql = Wiql(query="SELECT [System.Id] from WorkItems WHERE [System.AreaPath] under '"+area+"' AND [System.IterationPath] under 'Skanska Sverige IT' AND [System.WorkItemType] = 'Requirement'")
wiql_results = wit_client.query_by_wiql(wiql, top=10000).work_items

if wiql_results:       
    for item in wiql_results: desired_ids.append(int(item.id))
    pd.concat([all_tasks, pd.DataFrame(wiql_results) ])

len(all_tasks)

0

In [5]:

workitems_after_planning = list()
workitems_after_delivery= list()

start = 0 
stop = 0
max = len(desired_ids)-1
while stop<max:
    if (start+100)<=max: stop = start+99  
    else:                stop = stop+(len(desired_ids) - start)
    workitems_after_planning += wit_client.get_work_items(ids=desired_ids[start:stop], as_of=after_planning, error_policy="omit" )
    workitems_after_delivery    += wit_client.get_work_items(ids=desired_ids[start:stop], as_of=after_delivery, error_policy="omit" ) #2:00
    start+=100

# These items is then loaded from different time 

In [6]:
def getTasks(workitems):
    result = list()
    for item in workitems:
        if (item is None): continue
        result.append(
            {
               "id": item.id,
                "area": item.fields["System.AreaPath"],
                "iteration": item.fields["System.IterationPath"],
                "type": item.fields["Microsoft.VSTS.CMMI.RequirementType"],
                "title": item.fields["System.Title"],
                "state.start": item.fields["System.State"],
                "state.end": item.fields["System.State"],
                "estimate": item.fields["Microsoft.VSTS.Scheduling.OriginalEstimate"] if "Microsoft.VSTS.Scheduling.OriginalEstimate" in item.fields else "", 
                "activated" : item.fields["Microsoft.VSTS.Common.ActivatedDate"] if "Microsoft.VSTS.Common.ActivatedDate" in item.fields else "",
                "resolved": item.fields["Microsoft.VSTS.Common.ResolvedDate"] if "Microsoft.VSTS.Common.ResolvedDate" in item.fields else ""
            })  
    return result

In [7]:
req_after_planning = pd.DataFrame(getTasks(workitems_after_planning))
req_after_delivery = pd.DataFrame(getTasks(workitems_after_delivery))


In [8]:
req_after_planning #.info()

,id,area,iteration,type,title,state.start,state.end,estimate,activated,resolved
0,43801,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT\Archive\2022\Leveransperiod...,Planerat,Ny IRIS-rapport summerat lagervärde,Closed,Closed,40.0,2022-04-06T09:09:42.417Z,2022-05-03T14:52:50.66Z
1,44643,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT\Archive\2021\Leveransperiod...,Planerat,Försäljningsartiklar i webshop ska visas med m...,Closed,Closed,,2021-06-17T09:32:41.393Z,2021-09-27T10:25:59.807Z
2,45367,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT,Planerat,"Signera knapp under tillbehör, inte endast ski...",Closed,Closed,,2021-03-22T09:39:37.47Z,2021-03-23T07:30:32.51Z
3,45610,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT\Archive\2021\Leveransperiod...,Planerat,Vy för att följa försenade inleveranser,Closed,Closed,30.0,2021-03-25T10:53:49.873Z,2021-04-22T13:07:11.593Z
4,45684,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT,Planerat,Sammanfattning arbetad tid (SoT),Closed,Closed,4.0,2021-03-01T08:56:11.273Z,2021-03-01T08:56:11.273Z
...,...,...,...,...,...,...,...,...,...,...
2606,95458,Skanska Sverige IT\Removed PO\Global Services\...,Skanska Sverige IT\2023\Vinter 2023-4,Planerat,Corp Release 2023.08,Active,Active,12.0,2023-11-17T15:14:53.72Z,
2607,95462,Skanska Sverige IT\Removed PO\Global Services\...,Skanska Sverige IT\2023\Vinter 2023-4,Planerat,Remove support for old versions of Core services,Active,Active,8.0,2023-11-17T23:09:28.14Z,
2608,95466,Skanska Sverige IT\Removed PO\Global Services,Skanska Sverige IT\2023\Vinter 2023-4,Planerat,Bostad Release 2023.09,Active,Active,8.0,2023-11-18T00:28:19Z,
2609,95468,Skanska Sverige IT\ISO\Finans och Inkop,Skanska Sverige IT\2023\Höst 2023-3\Sprint 5,Planerat,[From Business Platform/IECC] Integration with...,Proposed,Proposed,,,


# The data is merged into one dataframe

In [9]:
df = pd.merge(req_after_planning,req_after_delivery , on="id", how="right") 
df['estimate_z'] = df['estimate_x'].where(df['estimate_x'].notnull(), df['estimate_y'])
df = df[['id', 'area_y', 'iteration_y', 'title_y','type_y','state.start_x', 'state.end_y', 'estimate_z', 'activated_y', 'resolved_y'  ]]
df = df.rename(columns={'area_y': 'area', 'iteration_y': 'iteration', 'title_y': 'title', 'state.start_x': 'state.start', 'state.end_y': 'state.end', 'type_y': 'type', 'estimate_z': 'estimate', 'activated_y': 'activated', 'resolved_y': 'resolved'})
df

,id,area,iteration,title,type,state.start,state.end,estimate,activated,resolved
0,43801,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT\Archive\2022\Leveransperiod...,Ny IRIS-rapport summerat lagervärde,Planerat,Closed,Closed,40.0,2022-04-06T09:09:42.417Z,2022-05-03T14:52:50.66Z
1,44643,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT\Archive\2021\Leveransperiod...,Försäljningsartiklar i webshop ska visas med m...,Planerat,Closed,Closed,,2021-06-17T09:32:41.393Z,2021-09-27T10:25:59.807Z
2,45367,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT,"Signera knapp under tillbehör, inte endast ski...",Planerat,Closed,Closed,,2021-03-22T09:39:37.47Z,2021-03-23T07:30:32.51Z
3,45610,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT\Archive\2021\Leveransperiod...,Vy för att följa försenade inleveranser,Planerat,Closed,Closed,30.0,2021-03-25T10:53:49.873Z,2021-04-22T13:07:11.593Z
4,45684,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT,Sammanfattning arbetad tid (SoT),Planerat,Closed,Closed,4.0,2021-03-01T08:56:11.273Z,2021-03-01T08:56:11.273Z
...,...,...,...,...,...,...,...,...,...,...
2815,99737,Skanska Sverige IT\ISO\Entreprenad,Skanska Sverige IT\2024,Littera/ID på Kalkylpost,Planerat,NaN,Proposed,30.0,,
2816,99738,Skanska Sverige IT\ISO\Entreprenad,Skanska Sverige IT\2024,Kalkylering i tidsaxel för att få fram mängd,Planerat,NaN,Proposed,150.0,,
2817,99743,Skanska Sverige IT\ISO\Entreprenad,Skanska Sverige IT\2024,Avisering i SPIK,Planerat,NaN,Proposed,200.0,,
2818,99754,Skanska Sverige IT\ISO\Styrning och Arkitektur,Skanska Sverige IT\2023\Vinter 2023-4,Se över objekt som saknar grupp i objektsbibli...,Planerat,NaN,Proposed,,,


In [10]:
df = df.replace("NaN", 0)
df['estimate'].replace('',0)
df['estimate'] = pd.to_numeric(df['estimate'])
df

,id,area,iteration,title,type,state.start,state.end,estimate,activated,resolved
0,43801,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT\Archive\2022\Leveransperiod...,Ny IRIS-rapport summerat lagervärde,Planerat,Closed,Closed,40.0,2022-04-06T09:09:42.417Z,2022-05-03T14:52:50.66Z
1,44643,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT\Archive\2021\Leveransperiod...,Försäljningsartiklar i webshop ska visas med m...,Planerat,Closed,Closed,NaN,2021-06-17T09:32:41.393Z,2021-09-27T10:25:59.807Z
2,45367,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT,"Signera knapp under tillbehör, inte endast ski...",Planerat,Closed,Closed,NaN,2021-03-22T09:39:37.47Z,2021-03-23T07:30:32.51Z
3,45610,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT\Archive\2021\Leveransperiod...,Vy för att följa försenade inleveranser,Planerat,Closed,Closed,30.0,2021-03-25T10:53:49.873Z,2021-04-22T13:07:11.593Z
4,45684,Skanska Sverige IT\Team\Maskinuthyrning,Skanska Sverige IT,Sammanfattning arbetad tid (SoT),Planerat,Closed,Closed,4.0,2021-03-01T08:56:11.273Z,2021-03-01T08:56:11.273Z
...,...,...,...,...,...,...,...,...,...,...
2815,99737,Skanska Sverige IT\ISO\Entreprenad,Skanska Sverige IT\2024,Littera/ID på Kalkylpost,Planerat,NaN,Proposed,30.0,,
2816,99738,Skanska Sverige IT\ISO\Entreprenad,Skanska Sverige IT\2024,Kalkylering i tidsaxel för att få fram mängd,Planerat,NaN,Proposed,150.0,,
2817,99743,Skanska Sverige IT\ISO\Entreprenad,Skanska Sverige IT\2024,Avisering i SPIK,Planerat,NaN,Proposed,200.0,,
2818,99754,Skanska Sverige IT\ISO\Styrning och Arkitektur,Skanska Sverige IT\2023\Vinter 2023-4,Se över objekt som saknar grupp i objektsbibli...,Planerat,NaN,Proposed,NaN,,


## Clean Data

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2820 entries, 0 to 2819
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           2820 non-null   int64  
 1   area         2820 non-null   object 
 2   iteration    2820 non-null   object 
 3   title        2820 non-null   object 
 4   type         2820 non-null   object 
 5   state.start  2611 non-null   object 
 6   state.end    2820 non-null   object 
 7   estimate     1963 non-null   float64
 8   activated    2820 non-null   object 
 9   resolved     2820 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 220.4+ KB


In [135]:
df = df[df["area"].str.contains("ARIS") == False]
df = df[df["area"].str.contains("Moved To HQ") == False]

df['area'].replace(to_replace = "^.*Global Services.*$", value = f"Global Services", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*Business Platforms.*$", value = f"Business Platforms", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*Digital arbetsplats.*$", value = f"Digital arbetsplats", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*Nordic Workplace.*$", value = f"Nordic Workplace", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*Service Desk.*$", value = f"Service Desk", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*HR.*$", value = f"HR", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*Digihub.*$", value = f"Digihub", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*Entreprenad.*$", value = f"Entreprenad", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*Finans.*$", value = f"Finans", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*Fusion.*$", value = f"Fusion", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*Information_data_management.*$", value = f"Information_data_management", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*Inkop.*$", value = f"Inkop", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*Marknad.*$", value = f"Marknad", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*Maskinuthyrning.*$", value = f"Maskinuthyrning", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*TUI.*$", value = f"TUI", inplace=True, regex=True)
df['area'].replace(to_replace = "^.*Uppdragsverksamhet.*$", value = f"Uppdragsverksamhet", inplace=True, regex=True)

df = df[df["area"] != "" ]
df = df[df["area"] != "Proposed"]
df = df[df["area"] != "Skanska Sverige IT"]



df.info()
#df

<class 'pandas.core.frame.DataFrame'>
Index: 2516 entries, 0 to 2702
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           2516 non-null   int64  
 1   area         2516 non-null   object 
 2   iteration    2516 non-null   object 
 3   title        2516 non-null   object 
 4   type         2516 non-null   object 
 5   state.start  2463 non-null   object 
 6   state.end    2516 non-null   object 
 7   estimate     1758 non-null   float64
 8   activated    2516 non-null   object 
 9   resolved     2516 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 216.2+ KB


# The dataframe is stored as csv into a file

In [12]:
df.to_csv("behov-vinter2023.csv", sep=";", decimal=',')